In [ ]:
from roler.model import ModelPrior, ModelParams
from roler.distributions import *

In [2]:
prior = ModelPrior(
    individuals_local=IntDistribution(100, 10**8),
    add_individuals_meta=IntDistribution(100, 10**8),
    
    prop_species_meta=FloatDistribution(0.0, 1.0),
    speciation_local=FloatDistribution(0.0001, 0.9999),
    speciation_meta=FloatDistribution(0.01, 1.0),
    
    extinction_meta=FloatDistribution(0.01, 1.0),
    env_sigma=FloatDistribution(0.0001, 1000.0),
    trait_sigma=FloatDistribution(0.0001, 1000.0),
    comp_sigma=FloatDistribution(0.0001, 1000.0),
    dispersal_prob=FloatDistribution(0.0, 1.0),
    mutation_rate=FloatDistribution(1e-7, 0.001),
    equilib_escape=FloatDistribution(0.0, 1.0),
    num_basepairs=250,

    alpha=FloatDistribution(1.0, 10**6),
    neut_delta=FloatDistribution(0.0, 1.0),
    env_comp_delta=FloatDistribution(0.0, 1.0),

    init_type='oceanic_island',
    niter=10000,
    niterTimestep=10,
)

In [3]:
sample = prior.sample()

In [4]:
params = prior.get_params(sample)
params

ModelParams(individuals_local=56255284, individuals_meta=146619012, species_meta=17806243, speciation_local=0.8931140303611755, speciation_meta=0.6636114716529846, extinction_meta=0.6968994140625, env_sigma=5.192379474639893, trait_sigma=343.6248779296875, comp_sigma=365.31329345703125, dispersal_prob=0.702155590057373, mutation_rate=0.00021096516866236925, equilib_escape=0.565222442150116, num_basepairs=250, alpha=426006.90625, neut_delta=0.966285228729248, env_comp_delta=0.7300561666488647, init_type='oceanic_island', niter=10000, niterTimestep=10)

In [7]:
print(params.model_dump_json(indent=4))

{
    "individuals_local": 56255284,
    "individuals_meta": 146619012,
    "species_meta": 17806243,
    "speciation_local": 0.8931140303611755,
    "speciation_meta": 0.6636114716529846,
    "extinction_meta": 0.6968994140625,
    "env_sigma": 5.192379474639893,
    "trait_sigma": 343.6248779296875,
    "comp_sigma": 365.31329345703125,
    "dispersal_prob": 0.702155590057373,
    "mutation_rate": 0.00021096516866236925,
    "equilib_escape": 0.565222442150116,
    "num_basepairs": 250,
    "alpha": 426006.90625,
    "neut_delta": 0.966285228729248,
    "env_comp_delta": 0.7300561666488647,
    "init_type": "oceanic_island",
    "niter": 10000,
    "niterTimestep": 10
}
